In [1]:
#| code-fold: true
from IPython.core.interactiveshell import InteractiveShell

# `ast_node_interactivity` is a setting that determines how the return value of the last line in a cell is displayed
# with `last_expr_or_assign`, the return value of the last expression is displayed unless it is assigned to a variable
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

There's an excellent blog post on why Pandas feels clunky for those coming from R:

<https://www.sumsar.net/blog/pandas-feels-clunky-when-coming-from-r/>

In Python, I find `ibis` to be a much more natural fit for those coming from R. It provides a similar syntax and functionality, making it easier to transition between the two languages.

In [2]:
import ibis

`_` in ibis is a special variable that refers to the last expression evaluated
this is useful for chaining operations or for using the result of the last expression in subsequent operations


In [3]:
from ibis import _

ibis.options.interactive = True

Here's the equivalent code in `ibis` for the example provided in the blog post:



In [4]:
df = ibis.read_csv("purchases.csv")

┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓
┃ country ┃ amount ┃ discount ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩
│ string  │ int64  │ int64    │
├─────────┼────────┼──────────┤
│ USA     │   2000 │       10 │
│ USA     │   3500 │       15 │
│ USA     │   3000 │       20 │
│ Canada  │    120 │       12 │
│ Canada  │    180 │       18 │
│ Canada  │   3100 │       21 │
│ UK      │    130 │       13 │
│ UK      │    160 │       16 │
│ UK      │    190 │       19 │
│ France  │    110 │       11 │
│ …       │      … │        … │
└─────────┴────────┴──────────┘

In [5]:
df.amount.sum()

┌───────┐
│ 17210 │
└───────┘

In [6]:
df.group_by("country").aggregate(total=_.amount.sum())

┏━━━━━━━━━━━┳━━━━━━━┓
┃ country   ┃ total ┃
┡━━━━━━━━━━━╇━━━━━━━┩
│ string    │ int64 │
├───────────┼───────┤
│ Germany   │   570 │
│ India     │   720 │
│ Spain     │   660 │
│ Japan     │   690 │
│ Italy     │   630 │
│ Brazil    │   460 │
│ Canada    │  3400 │
│ France    │   500 │
│ USA       │  8500 │
│ Australia │   600 │
│ …         │     … │
└───────────┴───────┘

In [7]:
(
    df
    .group_by("country")
    .aggregate(total=(_.amount - _.discount).sum())
)

┏━━━━━━━━━━━┳━━━━━━━┓
┃ country   ┃ total ┃
┡━━━━━━━━━━━╇━━━━━━━┩
│ string    │ int64 │
├───────────┼───────┤
│ Italy     │   567 │
│ Brazil    │   414 │
│ USA       │  8455 │
│ Australia │   540 │
│ Canada    │  3349 │
│ France    │   450 │
│ UK        │   432 │
│ Germany   │   513 │
│ India     │   648 │
│ Spain     │   594 │
│ …         │     … │
└───────────┴───────┘

In [8]:
(
    df
    .mutate(median=_.amount.median())
    .filter(_.amount <= _.median * 10)
    .group_by("country")
    .aggregate(total=(_.amount - _.discount).sum())
)

┏━━━━━━━━━┳━━━━━━━┓
┃ country ┃ total ┃
┡━━━━━━━━━╇━━━━━━━┩
│ string  │ int64 │
├─────────┼───────┤
│ USA     │  1990 │
│ Spain   │   594 │
│ Japan   │   621 │
│ Germany │   513 │
│ India   │   648 │
│ UK      │   432 │
│ Italy   │   567 │
│ Brazil  │   414 │
│ Canada  │   270 │
│ France  │   450 │
│ …       │     … │
└─────────┴───────┘

In [9]:
(
    df
    .join(
        df.group_by("country").aggregate(median=_.amount.median()),
        predicates=["country"]
    )
    .filter(_.amount <= _.median * 10)
    .group_by("country")
    .aggregate(total=(_.amount - _.discount).sum())
    .order_by("country")
)

┏━━━━━━━━━━━┳━━━━━━━┓
┃ country   ┃ total ┃
┡━━━━━━━━━━━╇━━━━━━━┩
│ string    │ int64 │
├───────────┼───────┤
│ Australia │   540 │
│ Brazil    │   414 │
│ Canada    │   270 │
│ France    │   450 │
│ Germany   │   513 │
│ India     │   648 │
│ Italy     │   567 │
│ Japan     │   621 │
│ Spain     │   594 │
│ UK        │   432 │
│ …         │     … │
└───────────┴───────┘